### Importing libraries

In [1]:
from binance import Client
from datetime import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from binance.spot import Spot

client = Spot()
serverTime=client.time()['serverTime']
# Get server timestamp
print(datetime.utcfromtimestamp(serverTime/1000).strftime('%Y-%m-%d %H:%M:%S'))

2023-08-31 16:54:38


### Binance API

In [3]:
binance_api=
binance_secret=

client=Client(api_key=binance_api,api_secret=binance_secret)

In [6]:
pd.DataFrame(client.get_aggregate_trades(symbol='BTCUSDT', startTime='1534130155000', endTime='1689515611522',limit=1000) )

,a,p,q,f,l,T,m,M
0,55802169,6332.04000000,0.00565800,62974326,62974326,1534130155935,True,True
1,55802170,6332.09000000,0.03334200,62974327,62974327,1534130160045,True,True
2,55802171,6332.10000000,0.01620500,62974328,62974328,1534130161772,True,True
3,55802172,6332.01000000,0.02568300,62974329,62974329,1534130163187,True,True
4,55802173,6332.01000000,0.04098400,62974330,62974330,1534130163532,True,True
...,...,...,...,...,...,...,...,...
995,55803164,6345.00000000,0.00720700,62975416,62975416,1534130498018,True,True
996,55803165,6344.50000000,0.21439100,62975417,62975417,1534130498018,True,True
997,55803166,6344.51000000,0.32164200,62975418,62975418,1534130498568,True,True
998,55803167,6346.25000000,2.25610900,62975419,62975419,1534130499040,False,True


Tickers to download (largest market cap on 01.02.2023)

In [7]:
tickers=['BTCUSDT','ETHUSDT','BNBUSDT','XRPUSDT','ADAUSDT','DOGEUSDT','MATICUSDT','LTCUSDT']
#не стал брать:'DOTUSDT','SHIBUSDT',SOLUSDT так как эти токены очень поздно начали торговаться

In [8]:
#первая дата торгов токеном на Binance
for i in tickers:
    a=client.get_symbol_ticker(symbol=i)
    b=client._get_earliest_valid_timestamp(i,'1d')
    print(a['symbol'],':\t',datetime.utcfromtimestamp(b/1000).strftime('%Y-%m-%d'))

BTCUSDT :	 2017-08-17
ETHUSDT :	 2017-08-17
BNBUSDT :	 2017-11-06
XRPUSDT :	 2018-05-04
ADAUSDT :	 2018-04-17
DOGEUSDT :	 2019-07-05
MATICUSDT :	 2019-04-26
LTCUSDT :	 2017-12-13


In [9]:
storage={}
for i in tickers:
    d1=datetime.now().timestamp()
    first_date=client._get_earliest_valid_timestamp(i,'1d')
    storage[f'{i}']=client.get_historical_klines(i,'1h',first_date)
    d2=datetime.now().timestamp()
    print(i,'is downloaded, took',round(d2-d1,2),'seconds')

BTCUSDT is downloaded, took 39.01 seconds
ETHUSDT is downloaded, took 40.26 seconds
BNBUSDT is downloaded, took 37.48 seconds
XRPUSDT is downloaded, took 32.24 seconds
ADAUSDT is downloaded, took 33.25 seconds
DOGEUSDT is downloaded, took 31.88 seconds
MATICUSDT is downloaded, took 32.75 seconds
LTCUSDT is downloaded, took 34.22 seconds


Daily returns

In [10]:
daily_Rt=pd.DataFrame()
for key in storage.keys():
    arr=storage[key]
    df=pd.DataFrame(arr,columns=['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time',\
                              'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',\
                              'Taker_buy_quote_asset_volume', 'Ignore'])
    df.drop(columns=['Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',\
                                  'Taker_buy_quote_asset_volume', 'Ignore'],inplace=True)
    df['Human_open_time']=df.Open_time.map(lambda x: datetime.utcfromtimestamp(x/1000).\
                                           strftime('%Y-%m-%d %H:%M:%S') )
    df['date']=df.Human_open_time.astype('datetime64[D]')
    
    df_fin=df.groupby('date').agg({'Open':'first','High':'max','Low':'min','Close':'last'})
    df_fin['Close_Rt']=(df_fin.Close.apply(float)/df_fin.Close.shift(1).apply(float)).apply(np.log)*100
    df_fin.dropna(inplace=True)
    daily_Rt[key]=df_fin['Close_Rt']

In [21]:
#daily_Rt.to_csv('daily_Rt.csv')

Price trajectories

In [16]:
all_coins=pd.DataFrame()
for key in storage.keys():
    arr=storage[key]
    df=pd.DataFrame(arr,columns=['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time',\
                              'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',\
                              'Taker_buy_quote_asset_volume', 'Ignore'])
    df.drop(columns=['Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',\
                                  'Taker_buy_quote_asset_volume', 'Ignore'],inplace=True)
    df['Human_open_time']=df.Open_time.map(lambda x: datetime.utcfromtimestamp(x/1000).\
                                           strftime('%Y-%m-%d %H:%M:%S') )
    df['date']=df.Human_open_time.astype('datetime64[D]')
    df_fin=df.groupby('date').agg({'Open':'first','High':'max','Low':'min','Close':'last'})
    all_coins[key]=df_fin.Close

In [38]:
all_coins.to_csv('all_coins.csv')